In [1]:
%load_ext autoreload
%autoreload 2
import mmcv
from mmdet3d.apis import inference_mono_3d_detector, init_model
from mmdet3d.registry import VISUALIZERS
from mmengine import load, dump
from mmdet3d.registry import VISUALIZERS
import cv2
import numpy as np
import sys
from sensus.tools.new_visualizer import draw_monodetection_labels, draw_monodetection_results
from sensus.tools.data_processor_old import DataProcessor
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [13]:
file = '000369'
dataset = 'dair'
if dataset == 'kitti':
    img_file = '/home/javier/datasets/kitti/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/kitti/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/kitti/training/calib/'+ file + '.txt'
    pitch = 0.0
elif dataset == 'dair':
    img_file = '/home/javier/datasets/DAIR-V2X/single-infrastructure-side-mmdet/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/DAIR-V2X/single-infrastructure-side-mmdet/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/DAIR-V2X/single-infrastructure-side-mmdet/training/calib/'+ file + '.txt'
    pitch = 0.2031

ann_file = '/home/javier/sensus-loci/sensus/notebooks/data.pkl'
cam_type = 'CAM_BACK'

In [14]:
def create_pkl(file_name, calib_file, cam_type = 'CAM_BACK', pkl_path = 'data.pkl'):
    # Get the camera to image matrix
    array = DataProcessor(None, calib_file).process_calib_file()['P2']

    # Remove the last 3 elements of the array
    indices_to_remove = [3, 7, 11]
    array = [array[i] for i in range(len(array)) if i not in indices_to_remove]

    # Reshape the array into a 3x3 matrix
    cam2img = [array[i:i+3] for i in range(0, len(array), 3)]

    # Create the data dictionary
    data = [{'images': {cam_type: {'img_path': f'{file_name}.png', 'cam2img': cam2img}}}]

    # Save data to the pkl file
    with open(pkl_path, "wb") as file:
        pickle.dump(data, file)

    print(f"Data saved to {pkl_path}")
create_pkl(file, calib_file, cam_type, ann_file)

Data saved to /home/javier/sensus-loci/sensus/notebooks/data.pkl


## Draw Labels

In [16]:
draw_monodetection_labels(img_file, calib_file, labels_file, num_cars=15, thickness=3, out_path='labels')

## Inference

In [10]:
config_file = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d_pretrained_kitty.py'
config_file = '/home/javier/sensus-loci/work_dirs/exp/good/sensus_config_bueno/sensus_config.py'

checkpoint_file = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d_pretrained_kitty/epoch_100.pth'
checkpoint_file = '/home/javier/sensus-loci/work_dirs/exp/good/sensus_config_bueno/epoch_100.pth'

config_file = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d_pretrained_kitty/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d_pretrained_kitty.py'
checkpoint_file = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d_pretrained_kitty/epoch_100.pth'

config_file = '/home/javier/sensus-loci/mmdetection3d/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_kitti-mono3d.py'
checkpoint_file = '/home/javier/sensus-loci/mmdetection3d/checkpoints/smoke_dla34_pytorch_dlaneck_gn-all_8x4_6x_kitti-mono3d_20210929_015553-d46d9bb0.pth'

device = 'cuda:1'
model_mmdet = init_model(config_file, checkpoint_file, device=device)
result = inference_mono_3d_detector(model_mmdet, img_file, ann_file, cam_type)

# init visualizer
visualizer_mmdet = VISUALIZERS.build(model_mmdet.cfg.visualizer)
visualizer_mmdet.dataset_meta = model_mmdet.dataset_meta

Loads checkpoint by local backend from path: /home/javier/sensus-loci/mmdetection3d/checkpoints/smoke_dla34_pytorch_dlaneck_gn-all_8x4_6x_kitti-mono3d_20210929_015553-d46d9bb0.pth


/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmdet3d.visualization.local_visualizer.Det3DLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


In [11]:
print(result.pred_instances_3d)

<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([2], device='cuda:1')
    scores_3d: tensor([0.3033], device='cuda:1')
    bboxes_3d: CameraInstance3DBoxes(
            tensor([[ 2.8010, -0.4806, 21.7401,  3.8909,  1.6409,  1.6649, -1.4379]],
               device='cuda:1'))
) at 0x7f92b5b59bb0>


## MMDet Viz

In [9]:
img = mmcv.imread(img_file)
img = mmcv.imconvert(img, 'bgr', 'rgb')
data_input = dict(img=img)
# show the results
visualizer_mmdet.add_datasample(
    'result',
    data_input,
    data_sample=result,
    draw_gt=False,
    show=True,
    wait_time=0,
    out_file='result_mmdet',
    pred_score_thr= 0.30,
    vis_task='mono_det')

/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:812: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(


## Sensus Viz

In [12]:
draw_monodetection_results(img_file, calib_file, result, score=0.3, thickness=3, out_path='result_bad', index_car=2)